<a href="https://colab.research.google.com/github/Minlee01/TechPing-AI/blob/develop/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
 chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

In [ ]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

selenium이용한 동적 크롤링 실행

In [ ]:
import json

# 웹 드라이버 설정
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)

# 웹사이트 열기
driver.get("http://www.koreabond.co.kr/main/common/common/dic.cmd")

# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)  # 최대 10초 대기

data = []

try:
    # 단어 목록 가져오기
    words = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.words > li > a")))

    for i in range(len(words)):
        try:
            # 단어 목록을 매번 다시 가져오기 (StaleElementReferenceException 방지)
            words = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.words > li > a")))
            word = words[i]  # 현재 단어 선택

            # 스크롤로 단어 위치 조정
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", word)

            # 단어 클릭
            word.click()
            time.sleep(1)  # 클릭 후 데이터 로드 대기

            # 제목과 내용 추출
            title = wait.until(EC.presence_of_element_located((By.ID, "title"))).text
            content = wait.until(EC.presence_of_element_located((By.ID, "content"))).text

            # 데이터 저장
            data.append({"title": title, "content": content})
            print(f"데이터 추가 확인: {data[-1]}")  # 추가된 데이터 출력

        except Exception as e:
            print(f"Error: {e}")
            continue

finally:
    # 브라우저 닫기
    driver.quit()

# 디버깅: 데이터 리스트 내용 확인
print("데이터 확인:", data)

# JSON 파일로 저장
file_path = "dictionary.json"
if data:  # 데이터가 비어 있지 않은 경우에만 저장
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
        print(f"JSON 파일 저장 완료: {file_path}")
else:
    print("데이터가 비어 있습니다. JSON 파일을 저장하지 않습니다.")

BeautifulSoup로 정적 크롤링 실행

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# URL 기본 구성
base_url = "https://www.fsc.go.kr/in090301/view"

# 데이터 저장 리스트
data = []

# dicId 범위 설정 (예: 1698부터 1925까지)
start_id = 1698
end_id = 1925

for dic_id in range(start_id, end_id + 1):
    try:
        # 현재 페이지 URL
        url = f"{base_url}?dicId={dic_id}&curPage=1"
        print(f"크롤링 중: {url}")

        # 페이지 요청
        response = requests.get(url)
        response.raise_for_status()

        # BeautifulSoup로 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 제목과 내용 크롤링
        title = soup.select_one("div.subject").get_text(strip=True) if soup.select_one("div.subject") else "제목 없음"
        content = soup.select_one("div.cont").get_text(strip=True) if soup.select_one("div.cont") else "내용 없음"

        # 데이터 저장
        data.append({
            "dicId": dic_id,
            "title": title,
            "content": content
        })
        print(f"추출 완료: {title}")

    except Exception as e:
        print(f"Error at dicId {dic_id}: {e}")
        continue

# JSON 파일 저장
file_path = "dictionary_v2.json"
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)
    print(f"JSON 파일 저장 완료: {file_path}")


전에 생성된 파일 합침

In [ ]:
file1 = "dictionary.json"
file2 = "dictionary_v2.json"

import json

def load_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []

def merge_dictionaries(file1, file2, output_file):
    # JSON 데이터 로드
    data1 = load_json(file1)
    data2 = load_json(file2)

    # 최종 데이터를 저장할 딕셔너리
    final_data = []
    # 중복 확인을 위한 제목 저장소
    title_set = set()

    # 첫 번째 JSON 데이터 추가
    for item in data1:
        title = item["title"]
        content = item.get("content", "")
        if title not in title_set:
            final_data.append({"title": title, "content": content})
            title_set.add(title)

    # 두 번째 JSON 데이터 추가 (중복 제외)
    for item in data2:
        title = item["title"]
        if title not in title_set:
            final_data.append({"title": title, "content": content})
            title_set.add(title)

    # JSON 파일로 저장
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(final_data, file, ensure_ascii=False, indent=4)
    print(f"{output_file} 파일이 생성되었습니다.")

# 실행
merge_dictionaries("dictionary.json", "dictionary_v2.json", "dictionary_final.json")